In [1]:
cd /home/grad/Desktop/pietro/denovo/finetuning_results_gba/

/home/grad/Desktop/pietro/denovo/finetuning_results_gba


In [2]:
import os
import sys
import json
import pickle
import logging
import numpy as np
import tensorflow as tf
from rdkit import Chem
from rdkit.Chem import MolFromSmiles, MolToSmiles, FilterCatalog
from rdkit.Chem.Scaffolds import MurckoScaffold
from sklearn.model_selection import train_test_split
from typing import List, Tuple, Optional
import re
from threading import Lock
import time
import random

# Importiamo le classi custom per permettere a Keras di caricare il modello
from tensorflow.keras.layers import Layer, Embedding, Input, LayerNormalization, MultiHeadAttention, Dropout, Dense
from tensorflow.keras.models import Model, load_model

# ====================================================================
# [ 1. CONFIGURAZIONE FINE-TUNING ]
# ====================================================================

class config:
    # --- Percorsi File Esistenti (Dal primo training) ---
    PRETRAINED_MODEL = "/home/grad/Desktop/pietro/denovo/2/final_hybrid_model.keras"
    VOCAB_PATH = "/home/grad/Desktop/pietro/denovo/2/vocab.json"
    SMARTS_RX_FILE = "/home/grad/Desktop/pietro/denovo/2/SMART_RX/smartsrx-main/smartsrx.json"

    # --- Percorsi Nuovi Dati ---
    SMILES_FILE = "/home/grad/Desktop/pietro/denovo/s4-for-de-novo-drug-design/s4_loro/gen_mio/eval_out_gba/train_gba.smi" 
    SAVE_DIR = "finetuning_results_gba"
    PRINT_EVERY = 100
    # --- Iperparametri Fine-Tuning ---
    # Usiamo un Learning Rate più basso (es. 1e-5 o 5e-6)
    LR = 1e-5 
    BATCH_SIZE = 64
    EPOCHS = 50
    MAX_LENGTH = 140 # Deve essere uguale a quella del modello originale
    AUGMENT_PROB = 0.1
    VALID_RATIO = 0.1
    L2_REG = 1e-4
    CURRICULUM_START_COMPLEXITY = 1
    STEPS_PER_EPOCH = 100
        # Iperparametri Modello
    EMBED_DIM = 512
    TRANSFORMER_LAYERS = 6
    TRANSFORMER_HEADS = 6
    FF_DIM = 2048
    DROPOUT_RATE = 0.10
    L2_REG = 1e-4
    
    # Training
    BATCH_SIZE = 10
    EPOCHS = 3
    MAX_LENGTH = 140         # Aumentato leggermente per ospitare 3 parti
    GRADIENT_CLIP = 1.0
    VALID_RATIO = 0.1
    
    # Curriculum & Augmentation
    CURRICULUM_START_COMPLEXITY = 10
    CURRICULUM_COMPLEXITY_STEP = 5
    LOSS_STABILITY_THRESHOLD = 0.01
    WARMUP_EPOCHS = 5
    AUGMENT_PROB = 0.1
    
    # Generazione
    TEMPERATURE = 1.0
    GEN_NUM = 10
    PRINT_EVERY = 100
    STEPS_PER_EPOCH = 1000
os.makedirs(config.SAVE_DIR, exist_ok=True)

logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
logger = logging.getLogger(__name__)

# ====================================================================
# [ 2. DEFINIZIONI CUSTOM (Necessarie per caricare il modello) ]
# ====================================================================

from tensorflow.keras.layers import Layer, Dense, Dropout, Embedding, LayerNormalization, MultiHeadAttention, Input
from tensorflow.keras.models import load_model, Model

# Loss
def smoothed_loss(y_true, y_pred):
    y_true_int = tf.cast(y_true, tf.int32)
    mask = tf.cast(tf.math.not_equal(y_true, 0), tf.float32)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true_int, logits=y_pred)
    return tf.reduce_sum(loss * mask) / (tf.reduce_sum(mask) + 1e-9)

# Layers
class DynamicPositionalEncoding(Layer):
    def __init__(self, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        
    def build(self, input_shape):
        max_len = config.MAX_LENGTH
        pos = np.arange(max_len)[:, np.newaxis]
        i = np.arange(self.embed_dim)[np.newaxis, :]
        angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(self.embed_dim))
        angle_rads = pos * angle_rates
        angle_rads[:, 0::2] = tf.math.sin(angle_rads[:, 0::2])
        angle_rads[:, 1::2] = tf.math.cos(angle_rads[:, 1::2])
        self.pos_encoding = tf.cast(angle_rads[np.newaxis, ...], dtype=tf.float32)
        
    def call(self, inputs):
        seq_len = tf.shape(inputs)[1]
        return inputs + self.pos_encoding[:, :seq_len, :]
        
    def get_config(self):
        return {**super().get_config(), "embed_dim": self.embed_dim}

class ImprovedTransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ffn_dim, rate=0.1, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ffn_dim = ffn_dim
        self.rate = rate
        
        self.mha = MultiHeadAttention(
            num_heads=num_heads, 
            key_dim=embed_dim, 
            kernel_regularizer=tf.keras.regularizers.l2(config.L2_REG)
        )
        self.ffn = tf.keras.Sequential([
            Dense(ffn_dim, activation="gelu", kernel_regularizer=tf.keras.regularizers.l2(config.L2_REG)),
            Dense(embed_dim, kernel_regularizer=tf.keras.regularizers.l2(config.L2_REG))
        ])
        self.ln1 = LayerNormalization(epsilon=1e-6)
        self.ln2 = LayerNormalization(epsilon=1e-6)
        self.d1 = Dropout(rate)
        self.d2 = Dropout(rate)

    def call(self, inputs, training=False, mask=None):
        seq_len = tf.shape(inputs)[1]
        causal_mask = tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
        attn_output = self.mha(inputs, inputs, attention_mask=causal_mask)
        out1 = self.ln1(inputs + self.d1(attn_output, training=training))
        ffn_output = self.ffn(out1)
        return self.ln2(out1 + self.d2(ffn_output, training=training))
        
    def get_config(self):
        return {**super().get_config(), "embed_dim": self.embed_dim, "num_heads": self.num_heads, "ffn_dim": self.ffn_dim, "rate": self.rate}

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, embed_dim, warmup_steps=10000):
        super().__init__()
        self.embed_dim = tf.cast(embed_dim, tf.float32)
        self.warmup_steps = tf.cast(warmup_steps, tf.float32)

    def __call__(self, step):
        step = tf.cast(step, tf.float32) + 1e-9
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
        return tf.math.rsqrt(self.embed_dim) * tf.math.minimum(arg1, arg2)

    def get_config(self):
        return {"embed_dim": float(self.embed_dim), "warmup_steps": float(self.warmup_steps)}

# ====================================================================
# [ 3. UTILITY CHIMICHE (Copiate dall'originale) ]
# ====================================================================


# ====================================================================
# [ 2. GESTIONE SMARTS-RX & SCAFFOLD ]
# ====================================================================

# Variabili Globali Catalogo
SMARTS_CATALOG = None

def initialize_smarts_catalog():
    """Carica il catalogo SMARTS-RX."""
    global SMARTS_CATALOG
    if SMARTS_CATALOG is not None: return

    try:
        with open(config.SMARTS_RX_FILE, "rt", encoding="utf-8") as f:
            data = json.load(f)
    except Exception as e:
        logger.error(f"Errore caricamento SMARTS JSON: {e}")
        sys.exit(1)

    catalog = FilterCatalog.FilterCatalog()
    count = 0
    for entry in data.get("data", []):
        name = entry.get("specific_type")
        smarts = entry.get("smarts")
        if name and smarts:
            pattern = Chem.MolFromSmarts(smarts)
            if pattern:
                catalog.AddEntry(FilterCatalog.FilterCatalogEntry(name, FilterCatalog.SmartsMatcher(pattern)))
                count += 1
    SMARTS_CATALOG = catalog
    logger.info(f"Catalogo SMARTS-RX caricato: {count} regole.")

def get_smarts_fingerprint(mol: Chem.Mol) -> List[str]:
    """Restituisce lista nomi gruppi funzionali presenti."""
    if not mol or SMARTS_CATALOG is None: return []
    matches = SMARTS_CATALOG.GetMatches(mol)
    # Set per unicità, sorted per ordine deterministico
    return sorted(list(set([m.GetDescription() for m in matches])))

def get_murcko_scaffold_tokens(mol: Chem.Mol) -> List[str]:
    """Estrae scaffold e lo tokenizza."""
    if not mol: return []
    try:
        scaffold = MurckoScaffold.GetScaffoldForMol(mol)
        smi = Chem.MolToSmiles(scaffold, canonical=True, isomericSmiles=False)
        if not smi: return []
        return robust_tokenize(smi)
    except:
        return []

# ====================================================================
# [ 3. HELPER CHIMICI BASE ]
# ====================================================================

def robust_tokenize(smiles: str) -> list:
    pattern = (
        r"(\[[^\[\]]{1,6}\]|"                 # atomi in parentesi quadre
        r"Br|Cl|Si|Na|Mg|Mn|Ca|Fe|Zn|Se|Li|K|Al|B|"  # elementi multi-char
        r"R[0-9]|r[0-9]|a[0-9]|"             # ring labels
        r"[A-Za-z0-9@+\-\\\/\(\)=#\$\.\%,])"  # singoli caratteri, incluso '%'
    )
    tokens = re.findall(pattern, smiles)
    stack = []
    for t in tokens:
        if t.startswith('['): stack.append(t)
        if t.endswith(']'):
            if not stack: return []
            stack.pop()
    return tokens

def validate_and_fix_smiles(smiles: str) -> str:
    try:
        mol = Chem.MolFromSmiles(smiles, sanitize=True)
        if mol is None: return None
        try: Chem.Kekulize(mol, clearAromaticFlags=True)
        except: pass
        return Chem.MolToSmiles(mol, canonical=True, isomericSmiles=False)
    except: return None

def randomize_smiles(smiles: str, num_versions: int = 1) -> List[str]:
    mol = Chem.MolFromSmiles(smiles)
    if not mol: return []
    res = []
    try:
        s = Chem.MolToSmiles(mol, doRandom=True, canonical=False)
        if s: res.append(s)
    except: pass
    return res
def compute_complexity_from_tokens(tokens: List[str]) -> int:
    smiles = ''.join(tokens)
    try:
        mol = Chem.MolFromSmiles(smiles)
        if not mol: return 999
        return Chem.GetSSSR(mol) + smiles.count('(')
    except: return 999
# ====================================================================
# [ 4. PREPARAZIONE DATI FINE-TUNING ]
# ====================================================================

def process_dataset(smiles_list, vocab):
    initialize_smarts_catalog()
    char2idx = {c: i for i, c in enumerate(vocab)}
    processed = []
    
    logger.info(f"Processando {len(smiles_list)} SMILES per fine-tuning...")
    for s in smiles_list:
        mol = Chem.MolFromSmiles(s)
        if not mol: continue
        
        target_s = Chem.MolToSmiles(mol, canonical=True, isomericSmiles=False)
        tokens_target = robust_tokenize(target_s)
        
        # Scaffold
        scaf_mol = MurckoScaffold.GetScaffoldForMol(mol)
        scaf_smi = Chem.MolToSmiles(scaf_mol, canonical=True)
        tokens_scaf = robust_tokenize(scaf_smi)
        
        # SMARTS
        tokens_smarts = get_smarts_fingerprint(mol)
        
        # Controllo se i token esistono nel vecchio vocabolario
        # Se un nuovo SMILES ha un atomo mai visto prima, verrà marcato come <UNK>
        processed.append((tokens_smarts, tokens_scaf, tokens_target))
    logger.info(f"DONE. Valid: {len(processed)}.")

    return processed

# ====================================================================
# [ 5. CLASSI THREAD-SAFE ]
# ====================================================================

class ThreadSafeIterator:
    def __init__(self, iterator):
        self.iterator = iterator
        self.lock = Lock()
    def __iter__(self): return self
    def __next__(self):
        with self.lock: return next(self.iterator)

def threadsafe_generator(func):
    def wrapper(*args, **kwargs): return ThreadSafeIterator(func(*args, **kwargs))
    return wrapper

# ====================================================================
# [ 6. GENERATORE CURRICULUM (3 PARTI) ]
# ====================================================================

class CurriculumSmilesGenerator:
    def __init__(self, processed_data, vocab: List[str]):
        self.char2idx = {c: i for i, c in enumerate(vocab)}
        self.idx2char = {i: c for c, i in self.char2idx.items()}
        self.original_data = []
        
        # data = (smarts, scaffold, target)
        for smarts, scaffold, target in processed_data:
            comp = compute_complexity_from_tokens(target)
            self.original_data.append(((smarts, scaffold, target), comp))
            
        valid_comps = [c for _, c in self.original_data if c != 999]
        self.max_complexity = max(valid_comps) if valid_comps else 0
        self.current_complexity = config.CURRICULUM_START_COMPLEXITY
        self.available_data = self._filter_data()
        # Per novelty check
        self.train_smiles = {''.join(t) for (_, _, t), _ in self.original_data}
        self.lock = Lock()
    
    def _filter_data(self):
        filtered = [dp for dp, c in self.original_data if c <= self.current_complexity]
        return filtered if filtered else [dp for dp, _ in self.original_data]
    
    def update_complexity(self, epoch: int, loss_diff: float = None):
        with self.lock:
            if loss_diff is not None and loss_diff < config.LOSS_STABILITY_THRESHOLD:
                self.current_complexity = min(self.current_complexity + config.CURRICULUM_COMPLEXITY_STEP, self.max_complexity)
            else:
                if epoch <= config.WARMUP_EPOCHS:
                    incr = int((self.max_complexity - config.CURRICULUM_START_COMPLEXITY) * (epoch / config.WARMUP_EPOCHS))
                    self.current_complexity = config.CURRICULUM_START_COMPLEXITY + incr
                else:
                    self.current_complexity = self.max_complexity
            
            self.available_data = self._filter_data()
            if not self.available_data:
                self.available_data = [dp for dp, _ in self.original_data]
                logger.warning("Reset available_data (fallback).")
    
    @threadsafe_generator
    def __call__(self):
        PAD_IDX = self.char2idx['<PAD>']
        UNK_IDX = self.char2idx.get('<UNK>', PAD_IDX)

        while True:
            inputs = np.full((config.BATCH_SIZE, config.MAX_LENGTH), PAD_IDX, dtype=np.int32)
            targets = np.full_like(inputs, PAD_IDX)
            
            for i in range(config.BATCH_SIZE):
                with self.lock:
                    try:
                        data_pair = random.choice(self.available_data)
                    except:
                        self.available_data = [dp for dp, _ in self.original_data]
                        data_pair = random.choice(self.available_data)
                
                # Unpacking 3 parti
                t_smarts, t_scaf, t_targ = data_pair

                # Augmentation solo sul target
                curr_target = t_targ
                if random.random() < config.AUGMENT_PROB:
                    try:
                        aug = randomize_smiles(''.join(t_targ), 1)
                        if aug:
                            tok = robust_tokenize(aug[0])
                            if tok: curr_target = tok
                    except: pass

                # STRUTTURA: [START] SMARTS [SEP] SCAFFOLD [SEP] TARGET [END]
                seq = (['<START>'] + 
                       t_smarts + ['<SEP>'] + 
                       t_scaf + ['<SEP>'] + 
                       curr_target + ['<END>'])
                
                padded_seq = (seq + ['<PAD>'] * config.MAX_LENGTH)[:config.MAX_LENGTH]
                indices = [self.char2idx.get(t, UNK_IDX) for t in padded_seq]
                
                inputs[i] = indices
                targets[i, :-1] = inputs[i][1:]
                targets[i, -1] = PAD_IDX
                
            yield inputs, targets

    def get_dataset(self):
        return tf.data.Dataset.from_generator(
            self.__call__,
            output_signature=(
                tf.TensorSpec(shape=(config.BATCH_SIZE, config.MAX_LENGTH), dtype=tf.int32),
                tf.TensorSpec(shape=(config.BATCH_SIZE, config.MAX_LENGTH), dtype=tf.int32)
            )
        ).prefetch(tf.data.AUTOTUNE)
# ====================================================================
# [ 8. MONITORAGGIO (Parsing Complesso) ]
# ====================================================================
from tensorflow.keras.callbacks import Callback, ModelCheckpoint

class CustomTensorBoard(tf.keras.callbacks.TensorBoard):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}; lr = self.model.optimizer.learning_rate
        logs['lr'] = lr(epoch).numpy() if isinstance(lr, tf.keras.optimizers.schedules.LearningRateSchedule) else lr.numpy()
        super().on_epoch_end(epoch, logs)

class EnhancedTrainingMonitor(Callback):
    def __init__(self, val_gen: CurriculumSmilesGenerator):
        super().__init__()
        self.val_gen = val_gen
        self.best_val_loss = np.inf; self.prev_val_loss = None

    def generate_sample(self, num: int):
        generated, valid = [], []
        PAD, START, END, SEP = [self.val_gen.char2idx[k] for k in ['<PAD>','<START>','<END>','<SEP>']]
        input_seq = np.full((1, config.MAX_LENGTH), PAD, dtype=np.int32)
        
        for _ in range(num):
            input_seq.fill(PAD); input_seq[0, 0] = START
            for t in range(1, config.MAX_LENGTH):
                logits = self.model(input_seq, training=False)[0, t-1]
                probs = tf.nn.softmax(logits / config.TEMPERATURE).numpy()
                if np.sum(probs) < 1e-6: break
                sampled = np.random.choice(len(probs), p=probs)
                input_seq[0, t] = sampled
                if sampled == END: break
            
            indices = input_seq[0].tolist()
            raw_tokens = [self.val_gen.idx2char[i] for i in indices if i not in {PAD, START}]
            
            # Parsing Logic: START [SMARTS] SEP [SCAFFOLD] SEP [TARGET] END
            # Dobbiamo trovare l'ultimo SEP per ottenere il target
            smi_str = ""
            try:
                # Conta quanti SEP ci sono
                sep_indices = [i for i, x in enumerate(raw_tokens) if x == '<SEP>']
                if len(sep_indices) >= 2:
                    # Prendi tutto dopo il SECONDO SEP
                    target_tokens = raw_tokens[sep_indices[1]+1:]
                    smi_str = "".join(target_tokens).split('<END>')[0]
                elif len(sep_indices) == 1:
                    # Fallback: forse ha saltato uno step
                    target_tokens = raw_tokens[sep_indices[0]+1:]
                    smi_str = "".join(target_tokens).split('<END>')[0]
                else:
                    smi_str = "".join(raw_tokens).split('<END>')[0]
            except: pass
            
            final = validate_and_fix_smiles(smi_str)
            if final: valid.append(final)
            generated.append(final or smi_str)
            
        return generated, valid

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        if self.prev_val_loss and 'val_loss' in logs:
            diff = (self.prev_val_loss - logs['val_loss']) / self.prev_val_loss
            self.val_gen.update_complexity(epoch, diff)
        if logs.get('val_loss', np.inf) < self.best_val_loss: self.best_val_loss = logs['val_loss']
        self.prev_val_loss = logs.get('val_loss')
        
        if (epoch + 1) % config.PRINT_EVERY == 0:
            gen, val = self.generate_sample(config.GEN_NUM)
            validity = len(val) / config.GEN_NUM
            novel = len([s for s in val if s not in self.val_gen.train_smiles])
            logger.info(f"\nEPOCH {epoch+1}: Validity {validity:.1%} | Novelty {novel}/{len(val)}")
            if val: logger.info(f"Sample: {val[0]}")

2026-01-30 20:06:09.949559: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-30 20:06:09.963258: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769799969.980070  295009 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769799969.985103  295009 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769799969.997926  295009 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:
if __name__ == "__main__":
    # 1. Caricamento Vocabolario Originale
    if not os.path.exists(config.VOCAB_PATH):
        logger.error("Vocabolario originale non trovato! Necessario per FT."); sys.exit(1)
    with open(config.VOCAB_PATH, "r") as f:
        vocab = json.load(f)
    
    # 2. Caricamento Modello con Oggetti Custom
    logger.info("Caricamento modello pre-trainato...")
    model = load_model(config.PRETRAINED_MODEL, custom_objects = {
    "DynamicPositionalEncoding": DynamicPositionalEncoding,
    "ImprovedTransformerBlock": ImprovedTransformerBlock,
    "CustomSchedule": CustomSchedule,
    "smoothed_loss": smoothed_loss,
})
    
    # Riduciamo il Learning Rate per il fine-tuning
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=config.LR),
        loss=smoothed_loss
    )

    # 3. Caricamento Nuovi Dati
    with open(config.SMILES_FILE, "r") as f:
        new_smiles = [l.strip() for l in f if l.strip()]
    
    processed_data = process_dataset(new_smiles, vocab)
    train_data, val_data = train_test_split(processed_data, test_size=config.VALID_RATIO)
    # 3. SETUP GENERATORI
    logger.info("⚙️ Inizializzazione Generatori...")
    train_gen = CurriculumSmilesGenerator(train_data, vocab)
    val_gen = CurriculumSmilesGenerator(val_data, vocab)


    # 5. AVVIO TRAINING
    logger.info("🔥 Inizio Training Loop...")
    
    callbacks = [
        CustomTensorBoard(log_dir=f"logs/run_loaded_{int(time.time())}"),
        EnhancedTrainingMonitor(val_gen),
        ModelCheckpoint("best_hybrid_model.keras", monitor="val_loss", save_best_only=True)
    ]
    try:
        model.fit(
            train_gen.get_dataset(),
            epochs=config.EPOCHS,
            steps_per_epoch=config.STEPS_PER_EPOCH,
            validation_data=val_gen.get_dataset(),
            validation_steps=max(1, len(val_data)//config.BATCH_SIZE),
            callbacks=callbacks
        )
        model.save("final_hybrid_model.keras")
        logger.info("🏆 Training Completato.")
    except KeyboardInterrupt:
        logger.warning("⚠️ Training interrotto dall'utente.")
        model.save("interrupted_hybrid.keras")

2026-01-30 20:06:12,445 [INFO] Caricamento modello pre-trainato...
I0000 00:00:1769799972.546087  295009 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9890 MB memory:  -> device: 0, name: NVIDIA RTX A2000 12GB, pci bus id: 0000:65:00.0, compute capability: 8.6
/home/grad/anaconda3/envs/smiles-transformer/lib/python3.10/site-packages/keras/src/layers/layer.py:395: UserWarning: `build()` was called on layer 'improved_transformer_block', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/home/grad/anaconda3/envs/smiles-transformer/lib/python3.10/site-packages/keras/src/layers/layer.py:395: UserWarning: `build()` was called on layer 'improved_transformer_block_1', however the layer does not have a `build()` metho

Epoch 1/3


I0000 00:00:1769799987.350648  295253 service.cc:152] XLA service 0x7f6ebc00f780 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1769799987.350677  295253 service.cc:160]   StreamExecutor device (0): NVIDIA RTX A2000 12GB, Compute Capability 8.6
2026-01-30 20:06:28.229265: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1769799990.018577  295253 cuda_dnn.cc:529] Loaded cuDNN version 90300
2026-01-30 20:06:33.076662: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_154', 288 bytes spill stores, 288 bytes spill loads

2026-01-30 20:06:33.655058: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_165', 

   1/1000 ━━━━━━━━━━━━━━━━━━━━ 13:24:30 48s/step - loss: 0.5414

I0000 00:00:1769800025.937579  295253 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1000/1000 ━━━━━━━━━━━━━━━━━━━━ 143s 95ms/step - loss: 0.3397 - val_loss: 0.3709 - lr: 1.0000e-05
Epoch 2/3
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 91s 91ms/step - loss: 0.1835 - val_loss: 0.4548 - lr: 1.0000e-05
Epoch 3/3
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 91s 91ms/step - loss: 0.1698 - val_loss: 0.5098 - lr: 1.0000e-05


2026-01-30 20:11:44,941 [INFO] 🏆 Training Completato.
